In [2]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from CMA_obj import CMA_opt
from PEPG_obj import PEPG_opt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from SPSA_obj import SPSA_opt
from Finite_diff_grad import FD_opt
from ADAM_opt import AdamOptimizer
from PSO_obj import PSO_opt
from scipy.interpolate import interp1d
from numpy import asarray
from numpy import savetxt
from NN_utils_IRIS import *
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import pandas as pd

#   Online Training of Neural Networks IRIS, Wine

- The NN class helper functions and training loop functions are defined in NN_utils, 

### Loading datasets
X is the input, Y the output

In [3]:
#Iris dataset
iris_df = pd.read_csv("data\\IRIS\\iris.csv")
# convert the last column 


iris_raw = iris_df.values

for i in range(len(iris_raw)):
    if iris_raw[i,-1] == 'Iris-setosa':
        iris_raw[i,-1] = 0
    elif iris_raw[i,-1] == 'Iris-versicolor':
        iris_raw[i,-1] = 1
    else:
        iris_raw[i,-1] = 2
        
iris_raw = iris_raw.astype(np.float32)
#remove the first column because it is just an index
iris_raw = iris_raw[:,1:]
#iris raw needs to be shuffled randomly because the data is ordered by class
np.random.shuffle(iris_raw)

# Convert to PyTorch tensors
X = torch.from_numpy(iris_raw[:, :-1])
Y = torch.from_numpy(iris_raw[:, -1]).unsqueeze(1)

# Create a single dataset
full_dataset = Custom_dataset(X, Y)

# Split into train and test sets, first set the size of the split
train_size = int(0.75 * len(full_dataset))
test_size = len(full_dataset) - train_size
# split into train and test sets using pytorch randomsplit
Iris_train, Iris_test = torch.utils.data.random_split(full_dataset, [train_size, test_size])

Iris_train_loader = torch.utils.data.DataLoader(dataset=Iris_train, batch_size=train_size, shuffle=True)
Iris_test_loader = torch.utils.data.DataLoader(dataset=Iris_test, batch_size=test_size, shuffle=False)

c:\Users\Admin\Desktop\PhD\simulation\simulation_python\learning_strategies\NN_utils_IRIS.py:93: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.features = torch.tensor(features, dtype=torch.float)
c:\Users\Admin\Desktop\PhD\simulation\simulation_python\learning_strategies\NN_utils_IRIS.py:94: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels, dtype=torch.long).squeeze()  # Convert and squeeze labels


In [64]:
len(Iris_test_loader)

1

In [8]:
#We Now create an instance of the NN class and move if to the GPU if available
n_neurons = 10
NN_IRIS = Neural_Net(input_size=4, hidden_size=n_neurons, n_classes=3)

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(NN_IRIS.parameters(), lr=0.01)

#training the full NN
n_epochs = 200
test_acc = train_pytorch_NN(NN_IRIS, n_epochs, Iris_train_loader, Iris_test_loader, loss, optimizer)

Using cuda device
Neural_Net(
  (NN_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=4, out_features=10, bias=True)
    (2): ReLU()
    (3): Linear(in_features=10, out_features=3, bias=True)
  )
)
Epoch [1/200], Step [1/1], Loss: 1.1196235418319702, Test Accuracy: 60.526315789473685%
Epoch [2/200], Step [1/1], Loss: 1.0772697925567627, Test Accuracy: 31.57894736842105%
Epoch [3/200], Step [1/1], Loss: 1.0481926202774048, Test Accuracy: 31.57894736842105%
Epoch [4/200], Step [1/1], Loss: 1.0262075662612915, Test Accuracy: 31.57894736842105%
Epoch [5/200], Step [1/1], Loss: 1.0095471143722534, Test Accuracy: 31.57894736842105%
Epoch [6/200], Step [1/1], Loss: 0.9973515272140503, Test Accuracy: 31.57894736842105%
Epoch [7/200], Step [1/1], Loss: 0.9883865118026733, Test Accuracy: 31.57894736842105%
Epoch [8/200], Step [1/1], Loss: 0.9811901450157166, Test Accuracy: 23.68421052631579%
Epoch [9/200], Step [1/1], Loss: 0.9743272662162781, Test Accurac

In [53]:
from torchsummary import summary

summary(NN_IRIS, input_size=(4, 1))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                    [-1, 4]               0
            Linear-2                   [-1, 10]              50
              ReLU-3                   [-1, 10]               0
            Linear-4                    [-1, 3]              33
Total params: 83
Trainable params: 0
Non-trainable params: 83
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [56]:
print("Number of parameters in the network: ",NN_IRIS.num_params)
print(NN_IRIS)

Number of parameters in the network:  83
Neural_Net(
  (NN_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=4, out_features=10, bias=True)
    (2): ReLU()
    (3): Linear(in_features=10, out_features=3, bias=True)
  )
)


In [9]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(test_acc)), y=test_acc, mode='lines', name='Full NN'))
fig.update_layout(template='plotly_white', width=400, height=400,margin=dict(l=20, r=20, t=20, b=20))


In [10]:
np.savetxt('data\\Results\\NN_training\\online_training\\IRIS\\BP_test_acc.csv', test_acc, delimiter=',')

In [48]:
# Training loop PEPG for MNIST: 


#NN_MNIST.reset_weights()
#NN_MNIST.NN_stack[0].requires_grad = True
n_epochs =100
NN_IRIS = Neural_Net(input_size=4, hidden_size=10, n_classes=3)
N_dim = NN_IRIS.count_parameters()
pop_size = 100

#specify we don't need the computation graph to keep track of the gradients, we will use pepg to update the weights
with torch.no_grad():
    for param in NN_IRIS.parameters():
        param.requires_grad = False
loss = nn.CrossEntropyLoss()
# learning parameters


init_pos = NN_IRIS.get_params()

if init_pos.requires_grad:
    # Detach the tensor from the computation graph
    init_pos = init_pos.detach()
if init_pos.is_cuda:
    # Move the tensor to the CPU
    init_pos = init_pos.cpu()
init_pos = init_pos.numpy()

PEPG_optimizer = PEPG_opt(N_dim, pop_size, learning_rate=0.01, starting_mu=init_pos ,starting_sigma=0.1)

PEPG_optimizer.sigma_decay = 0.9999
PEPG_optimizer.sigma_alpha=0.2
PEPG_optimizer.sigma_limit=0.02
PEPG_optimizer.elite_ratio=0.1
PEPG_optimizer.weight_decay=0.005

test_acc_PEPG,best_reward_PEPG = train_online_pop_NN(NN_IRIS, n_epochs, Iris_train_loader, Iris_test_loader, loss, PEPG_optimizer)

Using cuda device
Neural_Net(
  (NN_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=4, out_features=10, bias=True)
    (2): ReLU()
    (3): Linear(in_features=10, out_features=3, bias=True)
  )
)
{i+1}Epoch [1/100], Step [1/1], Loss: 1.0891473293304443, Test Accuracy: 31.57894736842105%
{i+1}Epoch [2/100], Step [1/1], Loss: 1.0698035955429077, Test Accuracy: 13.157894736842104%
{i+1}Epoch [3/100], Step [1/1], Loss: 1.005279779434204, Test Accuracy: 76.3157894736842%
{i+1}Epoch [4/100], Step [1/1], Loss: 0.9963037371635437, Test Accuracy: 44.73684210526316%
{i+1}Epoch [5/100], Step [1/1], Loss: 1.052076816558838, Test Accuracy: 13.157894736842104%
{i+1}Epoch [6/100], Step [1/1], Loss: 1.0516226291656494, Test Accuracy: 31.57894736842105%
{i+1}Epoch [7/100], Step [1/1], Loss: 1.0145341157913208, Test Accuracy: 68.42105263157895%
{i+1}Epoch [8/100], Step [1/1], Loss: 1.0477441549301147, Test Accuracy: 2.6315789473684212%
{i+1}Epoch [9/100], Step [

In [38]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(test_acc_PEPG)), y=test_acc_PEPG, mode='lines', name='PEPG'))
#change theme to white and set the sizer of the plot
fig.update_layout(template='plotly_white', width=400, height=300,margin=dict(l=20, r=20, t=20, b=20))
fig.update_xaxes(title_text="Epochs",type = 'log')
fig.update_yaxes(title_text="Accuracy [%]")

In [13]:
#save pepg data
savetxt('data\\Results\\NN_training\\online_training\\IRIS\\PEPG_test_acc.csv', test_acc_PEPG, delimiter=',')

### We use CMA to train the FFNN
- This doesn't work at all this simple architecture has too many parameters ... so CMA is painfully slow.

In [46]:
#Using CMA-ES for training the NN
n_epochs =100
NN_IRIS = Neural_Net(input_size=4, hidden_size=10, n_classes=3)
N_dim = NN_IRIS.count_parameters()
pop_size = 50
#specify we don't need the computation graph to keep track of the gradients, we will use CMAES to update the weights
with torch.no_grad():
    for param in NN_IRIS.parameters():
        param.requires_grad = False
loss = nn.CrossEntropyLoss()
# learning parameters

init_pos = NN_IRIS.get_params()

if init_pos.requires_grad:
    # Detach the tensor from the computation graph
    init_pos = init_pos.detach()
if init_pos.is_cuda:
    # Move the tensor to the CPU
    init_pos = init_pos.cpu()
init_pos = init_pos.numpy()

CMA_optimizer = CMA_opt(N_dim, pop_size, select_pop=int(pop_size/2), sigma_init=0.1, mean_init=init_pos)
CMA_optimizer.eigen_update_frequency = 10

test_acc_CMA,best_reward_CMA = train_online_pop_NN(NN_IRIS, n_epochs, Iris_train_loader, Iris_test_loader, loss, CMA_optimizer)


Using cuda device
Neural_Net(
  (NN_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=4, out_features=10, bias=True)
    (2): ReLU()
    (3): Linear(in_features=10, out_features=3, bias=True)
  )
)
{i+1}Epoch [1/100], Step [1/1], Loss: 1.340676188468933, Test Accuracy: 23.68421052631579%
{i+1}Epoch [2/100], Step [1/1], Loss: 1.2666423320770264, Test Accuracy: 28.94736842105263%
{i+1}Epoch [3/100], Step [1/1], Loss: 1.15496027469635, Test Accuracy: 23.68421052631579%
{i+1}Epoch [4/100], Step [1/1], Loss: 1.144727349281311, Test Accuracy: 23.68421052631579%
{i+1}Epoch [5/100], Step [1/1], Loss: 1.1428987979888916, Test Accuracy: 31.57894736842105%
{i+1}Epoch [6/100], Step [1/1], Loss: 1.0938888788223267, Test Accuracy: 23.68421052631579%
{i+1}Epoch [7/100], Step [1/1], Loss: 1.1018266677856445, Test Accuracy: 23.68421052631579%
{i+1}Epoch [8/100], Step [1/1], Loss: 1.0667814016342163, Test Accuracy: 31.57894736842105%
{i+1}Epoch [9/100], Step [1/1]

In [47]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(test_acc_CMA)), y=test_acc_CMA, mode='lines', name='PEPG'))
#change theme to white and set the sizer of the plot
fig.update_layout(template='plotly_white', width=400, height=300,margin=dict(l=20, r=20, t=20, b=20))
fig.update_xaxes(title_text="Epochs",type = 'log')
fig.update_yaxes(title_text="Accuracy [%]")

In [45]:
savetxt('data\\Results\\NN_training\\online_training\\IRIS\\CMA_test_acc.csv', test_acc_CMA, delimiter=',')

In [43]:
#use SPSA to optimize The Neural network
n_epochs =500
NN_IRIS = Neural_Net(input_size=4, hidden_size=10, n_classes=3)
N_dim = NN_IRIS.count_parameters()
#specify we don't need the computation graph to keep track of the gradients, we will use SPSA to update the weights
with torch.no_grad():
    for param in NN_IRIS.parameters():
        param.requires_grad = False
loss = nn.CrossEntropyLoss()
# learning parameters

init_pos = NN_IRIS.get_params()

if init_pos.requires_grad:
    # Detach the tensor from the computation graph
    init_pos = init_pos.detach()
if init_pos.is_cuda:
    # Move the tensor to the CPU
    init_pos = init_pos.cpu()
init_pos = init_pos.numpy()

SPSA_optimizer = SPSA_opt(init_pos,alpha=1e-3,epsilon=1e-5)
Adam = AdamOptimizer(init_pos, lr=1e-2, beta1=0.9, beta2=0.99, epsilon=1e-8)

test_acc_SPSA, best_reward_SPSA = train_online_SPSA_NN(NN_IRIS, n_epochs, Iris_train_loader, Iris_test_loader, loss, SPSA_optimizer,Adam)

Using cuda device
Neural_Net(
  (NN_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=4, out_features=10, bias=True)
    (2): ReLU()
    (3): Linear(in_features=10, out_features=3, bias=True)
  )
)
{i+1}Epoch [1/500], Step [1/1], Loss: 1.1367748975753784, Test Accuracy: 44.73684210526316%
{i+1}Epoch [2/500], Step [1/1], Loss: 1.1318310499191284, Test Accuracy: 44.73684210526316%
{i+1}Epoch [3/500], Step [1/1], Loss: 1.1239755153656006, Test Accuracy: 44.73684210526316%
{i+1}Epoch [4/500], Step [1/1], Loss: 1.1196279525756836, Test Accuracy: 44.73684210526316%
{i+1}Epoch [5/500], Step [1/1], Loss: 1.1148799657821655, Test Accuracy: 44.73684210526316%
{i+1}Epoch [6/500], Step [1/1], Loss: 1.1067562103271484, Test Accuracy: 44.73684210526316%
{i+1}Epoch [7/500], Step [1/1], Loss: 1.0997792482376099, Test Accuracy: 44.73684210526316%
{i+1}Epoch [8/500], Step [1/1], Loss: 1.0949136018753052, Test Accuracy: 47.36842105263158%
{i+1}Epoch [9/500], Step [

In [44]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(test_acc_SPSA)), y=test_acc_SPSA, mode='lines', name='PEPG'))
#change theme to white and set the sizer of the plot
fig.update_layout(template='plotly_white', width=400, height=300,margin=dict(l=20, r=20, t=20, b=20))
fig.update_xaxes(title_text="Epochs",type = 'log')
fig.update_yaxes(title_text="Accuracy [%]")

In [49]:
savetxt('data\\Results\\NN_training\\online_training\\IRIS\\SPSA_test_acc.csv', test_acc_SPSA, delimiter=',')

In [59]:
#use FD to optimize The Neural network
n_epochs =1000
NN_IRIS = Neural_Net(input_size=4, hidden_size=10, n_classes=3)
N_dim = NN_IRIS.count_parameters()
grad_dim = 1

#specify we don't need the computation graph to keep track of the gradients, we will use SPSA to update the weights
with torch.no_grad():
    for param in NN_IRIS.parameters():
        param.requires_grad = False
loss = nn.CrossEntropyLoss()
# learning parameters

init_pos = NN_IRIS.get_params()

if init_pos.requires_grad:
    # Detach the tensor from the computation graph
    init_pos = init_pos.detach()
if init_pos.is_cuda:
    # Move the tensor to the CPU
    init_pos = init_pos.cpu()
init_pos = init_pos.numpy()

FD_optimizer = FD_opt(init_pos,n_perturb=grad_dim,alpha=1e-3,epsilon=1e-5)
Adam = AdamOptimizer(init_pos, lr=1e-2, beta1=0.9, beta2=0.99, epsilon=1e-8)

test_acc_FD, best_reward_FD = train_online_FD_NN(NN_IRIS,N_dim, n_epochs,  Iris_train_loader, Iris_test_loader, loss, FD_optimizer,Adam)

Using cuda device
Neural_Net(
  (NN_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=4, out_features=10, bias=True)
    (2): ReLU()
    (3): Linear(in_features=10, out_features=3, bias=True)
  )
)
{i+1}Epoch [1/1000], Step [1/1], Loss: 1.3367279767990112, Test Accuracy: 68.42105263157895%
{i+1}Epoch [2/1000], Step [1/1], Loss: 1.3356982469558716, Test Accuracy: 68.42105263157895%
{i+1}Epoch [3/1000], Step [1/1], Loss: 1.3340157270431519, Test Accuracy: 68.42105263157895%
{i+1}Epoch [4/1000], Step [1/1], Loss: 1.3326414823532104, Test Accuracy: 68.42105263157895%
{i+1}Epoch [5/1000], Step [1/1], Loss: 1.331472396850586, Test Accuracy: 68.42105263157895%
{i+1}Epoch [6/1000], Step [1/1], Loss: 1.3304648399353027, Test Accuracy: 68.42105263157895%
{i+1}Epoch [7/1000], Step [1/1], Loss: 1.3297957181930542, Test Accuracy: 68.42105263157895%
{i+1}Epoch [8/1000], Step [1/1], Loss: 1.3283112049102783, Test Accuracy: 68.42105263157895%
{i+1}Epoch [9/1000]

In [60]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(test_acc_FD)), y=test_acc_FD, mode='lines', name='PEPG'))
#change theme to white and set the sizer of the plot
fig.update_layout(template='plotly_white', width=400, height=300,margin=dict(l=20, r=20, t=20, b=20))
fig.update_xaxes(title_text="Epochs",type = 'log')
fig.update_yaxes(title_text="Accuracy [%]")

In [71]:
savetxt('data\\Results\\NN_training\\online_training\\IRIS\\FD-1_test_acc.csv', test_acc_FD, delimiter=',')

In [70]:
#Using PSO for training the NN
n_epochs =100
NN_IRIS = Neural_Net(input_size=4, hidden_size=10, n_classes=3)
N_dim = NN_IRIS.count_parameters()
pop_size = 50
#specify we don't need the computation graph to keep track of the gradients, we will use CMAES to update the weights
with torch.no_grad():
    for param in NN_IRIS.parameters():
        param.requires_grad = False
loss = nn.CrossEntropyLoss()
# learning parameters

init_pos = NN_IRIS.get_params()

if init_pos.requires_grad:
    # Detach the tensor from the computation graph
    init_pos = init_pos.detach()
if init_pos.is_cuda:
    # Move the tensor to the CPU
    init_pos = init_pos.cpu()
init_pos = init_pos.numpy()

#params dictionary
upper_bound = 0.2
lower_bound = -0.2

params = {'c_1': 2, 
          'c_2': 1,
          'w': 0.7,
          'Vmax': 0.15*(upper_bound-lower_bound),
          'upper_bound': upper_bound,
          'lower_bound': lower_bound,
          'pop_size' :pop_size,
          }

init_pos = (upper_bound - lower_bound) * np.random.rand(N_dim, pop_size) + lower_bound
V_init = 0.1 * np.random.rand(N_dim, pop_size)
PSO_optimizer = PSO_opt(X_init = init_pos,V_init = V_init,params=params)

test_acc_PSO,best_reward_PSO = train_online_pop_NN(NN_IRIS, n_epochs,  Iris_train_loader, Iris_test_loader, loss, PSO_optimizer)

Using cuda device
Neural_Net(
  (NN_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=4, out_features=10, bias=True)
    (2): ReLU()
    (3): Linear(in_features=10, out_features=3, bias=True)
  )
)
{i+1}Epoch [1/100], Step [1/1], Loss: 1.1198173761367798, Test Accuracy: 34.21052631578947%
{i+1}Epoch [2/100], Step [1/1], Loss: 1.0918654203414917, Test Accuracy: 31.57894736842105%
{i+1}Epoch [3/100], Step [1/1], Loss: 1.0703763961791992, Test Accuracy: 31.57894736842105%
{i+1}Epoch [4/100], Step [1/1], Loss: 1.0582610368728638, Test Accuracy: 31.57894736842105%
{i+1}Epoch [5/100], Step [1/1], Loss: 1.0257458686828613, Test Accuracy: 50.0%
{i+1}Epoch [6/100], Step [1/1], Loss: 0.9863336682319641, Test Accuracy: 36.8421052631579%
{i+1}Epoch [7/100], Step [1/1], Loss: 0.953170895576477, Test Accuracy: 68.42105263157895%
{i+1}Epoch [8/100], Step [1/1], Loss: 0.915444552898407, Test Accuracy: 76.3157894736842%
{i+1}Epoch [9/100], Step [1/1], Loss: 0.891

In [72]:
savetxt('data\\Results\\NN_training\\online_training\\IRIS\\PSO_test_acc.csv', test_acc_PSO, delimiter=',')

In [67]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(test_acc_PSO)), y=test_acc_PSO, mode='lines', name='PEPG'))
#change theme to white and set the sizer of the plot
fig.update_layout(template='plotly_white', width=400, height=300,margin=dict(l=20, r=20, t=20, b=20))
fig.update_xaxes(title_text="Epochs",type = 'log')
fig.update_yaxes(title_text="Accuracy [%]")

In [ ]:
model = NN_MNIST

# Initialize a list to store the figures
figs = []

# Iterate through each model parameter
for name, param in model.named_parameters():
    if 'weight' in name:  # Filter out only weight parameters
        # Flatten the weights
        weights = param.detach().cpu().numpy().flatten()
        
        # Create a histogram for the weights
        fig = go.Figure()
        fig.add_trace(go.Histogram(x=weights, name=name))
        
        # Update layout to add titles and improve readability
        fig.update_layout(
            title=f'Histogram of Weights for Layer: {name}',
            xaxis_title='Weight values',
            yaxis_title='Frequency',
            bargap=0.2
        )
        
        # Append the figure to the list
        figs.append(fig)

# Show all histograms
for fig in figs:
    fig.show()

In [ ]:

# Wine dataset
wine_df = pd.read_csv("data\\WINE\\winequality-red.csv")

wine_raw = wine_df.values.astype(np.float32)

# Convert to PyTorch tensors
X = torch.from_numpy(wine_raw[:, :-1])
Y = torch.from_numpy(wine_raw[:, -1]).unsqueeze(1)

# Create a single dataset
full_dataset = Custom_dataset(X, Y)

# Split into train and test sets, first set the size of the split
train_size = int(0.75 * len(full_dataset))
test_size = len(full_dataset) - train_size
# split into train and test sets using pytorch randomsplit

Wine_train, Wine_test = torch.utils.data.random_split(full_dataset, [train_size, test_size])


Wine_train_loader = torch.utils.data.DataLoader(dataset=Wine_train, batch_size=100, shuffle=True)
Wine_test_loader = torch.utils.data.DataLoader(dataset=Wine_test, batch_size=100, shuffle=False)
